In [2]:
!pip3 install --quiet torch
!pip3 install --quiet -q transformers datasets
!pip3 install --quiet --upgrade scikit-learn==1.0.2
!pip3 install --quiet matplotlib
!pip3 install --quiet accelerate -U
!pip install --quiet datasets -q
!pip install --quiet wordcloud -q
!pip install --quiet sentence-transformers -q
!pip install --quiet nltk 

In [3]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import re
import pandas as pd
import  matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from wordcloud import WordCloud
from datasets import load_dataset
from wordcloud import WordCloud

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type != 'cuda':
    raise SystemError('GPU device not found')

In [5]:
dataset = load_dataset("yelp_review_full")
train_dataset = dataset['train']
test_dataset = dataset['test']
del dataset

train_text = [train_dataset[i]['text'] for i in range(len(train_dataset))]
train_label = [train_dataset[i]['label'] for i in range(len(train_dataset))]

test_text = [test_dataset[i]['text'] for i in range(len(test_dataset))]
test_label = [test_dataset[i]['label'] for i in range(len(test_dataset))]


del train_dataset
del test_dataset

In [6]:
tokenizer = AutoTokenizer.from_pretrained("rttl-ai/bert-base-uncased-yelp-reviews")
model = AutoModelForSequenceClassification.from_pretrained("rttl-ai/bert-base-uncased-yelp-reviews")
model = model.to(device)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
@torch.no_grad()
def predict(text , tokenizer, model):
    input_encodings = tokenizer(text, return_tensors='pt', padding=True , truncation=True, max_length=512).to(device)
    outputs = model(**input_encodings)

    pred_label = torch.argmax(outputs.logits).item()

    return pred_label

In [10]:
y_pred  = []

for txt in tqdm(test_text):
    y_pred.append(predict(txt))


100%|██████████| 50000/50000 [11:15<00:00, 73.97it/s]


In [11]:
print(classification_report(test_label, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81     10000
           1       0.65      0.68      0.66     10000
           2       0.69      0.64      0.66     10000
           3       0.65      0.62      0.64     10000
           4       0.77      0.81      0.79     10000

    accuracy                           0.71     50000
   macro avg       0.71      0.71      0.71     50000
weighted avg       0.71      0.71      0.71     50000



In [12]:
print(confusion_matrix(test_label, y_pred))

[[8229 1665   72    8   26]
 [1836 6793 1307   48   16]
 [ 159 1896 6380 1449  116]
 [  24   70 1409 6186 2311]
 [  31   12  108 1779 8070]]


In [13]:
tokenizer2 = AutoTokenizer.from_pretrained("JariYP/finetuned_yelp")
model2 = AutoModelForSequenceClassification.from_pretrained("JariYP/finetuned_yelp")
model2 = model2.to(device)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at JariYP/finetuned_yelp and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
y_pred  = []

for txt in tqdm(test_text):
    y_pred.append(predict(txt , tokenizer2,model2))


100%|██████████| 50000/50000 [04:35<00:00, 181.35it/s]


In [17]:
print(classification_report(test_label, y_pred))

              precision    recall  f1-score   support

           0       0.20      0.88      0.33     10000
           1       0.16      0.12      0.13     10000
           2       0.00      0.00      0.00     10000
           3       0.00      0.00      0.00     10000
           4       0.00      0.00      0.00     10000

    accuracy                           0.20     50000
   macro avg       0.07      0.20      0.09     50000
weighted avg       0.07      0.20      0.09     50000



/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta